In [ ]:
!pip install transformers

In [ ]:
from transformers import BertModel, BertTokenizer, get_linear_schedule_with_warmup
from torch.optim import AdamW
import torch
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from textwrap import wrap

In [ ]:
#INICIALIZARON
RANDOM_SEED = 42
MAX_LEN = 150
BATCH_SIZE = 10
NCLASSES = 3

In [ ]:
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
df = pd.read_csv("Data_Moderador_IA.csv")
df_destil = df[:1500]
print(f'Columnas: {df_destil.columns}')
print(df_destil)

Columnas: Index(['id', 'Text', 'Label', 'language'], dtype='object')
          id                                         Text  Label language
0        1.0                  "bro tus ideas dan hueva 😴"  toxic  español
1        2.0                           "aprende a pensar"  toxic  español
2        3.0  "Perverted faggots stay away from children"   hate   ingles
3        4.0                      "mejor guarda silencio"  toxic  español
4        5.0                "cuando es el proximo match?"  clean  español
...      ...                                          ...    ...      ...
1495  1496.0                        "JAJAJA ese final XD"  clean  español
1496  1497.0                          "hablas por hablar"  toxic  español
1497  1498.0                       "q ganas d quedar mal"  toxic  español
1498  1499.0                           "cansas demasiado"  toxic  español
1499  1500.0                             "nooo pero bn 👍"  clean  español

[1500 rows x 4 columns]


In [ ]:
#REAJUSTE DE DATA
df_destil['Label'] = df_destil['Label'].map({
    'clean' : 0,
    'toxic' : 1,
    'hate' : 2
 })
df_destil.head()

/tmp/ipython-input-2662691084.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_destil['Label'] = df_destil['Label'].map({


,id,Text,Label,language
0,1.0,"""bro tus ideas dan hueva 😴""",1,español
1,2.0,"""aprende a pensar""",1,español
2,3.0,"""Perverted faggots stay away from children""",2,ingles
3,4.0,"""mejor guarda silencio""",1,español
4,5.0,"""cuando es el proximo match?""",0,español


In [ ]:
print(df_destil)

          id                                         Text  Label language
0        1.0                  "bro tus ideas dan hueva 😴"      1  español
1        2.0                           "aprende a pensar"      1  español
2        3.0  "Perverted faggots stay away from children"      2   ingles
3        4.0                      "mejor guarda silencio"      1  español
4        5.0                "cuando es el proximo match?"      0  español
...      ...                                          ...    ...      ...
1495  1496.0                        "JAJAJA ese final XD"      0  español
1496  1497.0                          "hablas por hablar"      1  español
1497  1498.0                       "q ganas d quedar mal"      1  español
1498  1499.0                           "cansas demasiado"      1  español
1499  1500.0                             "nooo pero bn 👍"      0  español

[1500 rows x 4 columns]


In [ ]:
#TOKENIZACION
PRE_TRAINED_MODEL_NAME = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
#EJEMPLO TOKENIZACION
simple_tex = "bro eres un pendejo 😴"
tokens = tokenizer.tokenize(simple_tex)
tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
print('frase', simple_tex)
print('tokens', tokens)
print('tokens numericos', tokens_ids)

frase bro eres un pendejo 😴
tokens ['bro', 'ere', '##s', 'un', 'pen', '##dej', '##o', '[UNK]']
tokens numericos [93742, 14378, 10107, 10119, 66558, 90062, 10133, 100]


In [ ]:
#CODIFICACION PARA INTRUDUCIR BERTH
encoding = tokenizer.encode_plus(
    simple_tex,
    max_length = 15,
    truncation = True,
    add_special_tokens = True,
    return_token_type_ids = False,
    padding = 'max_length',
    return_attention_mask = True,
    return_tensors = 'pt'
)

In [ ]:
encoding.keys()

KeysView({'input_ids': tensor([[  101, 93742, 14378, 10107, 10119, 66558, 90062, 10133,   100,   102,
             0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]])})

In [ ]:
print(tokenizer.convert_ids_to_tokens(encoding['input_ids'][0]))
print(encoding['input_ids'][0])
print(encoding['attention_mask'][0])

['[CLS]', 'bro', 'ere', '##s', 'un', 'pen', '##dej', '##o', '[UNK]', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
tensor([  101, 93742, 14378, 10107, 10119, 66558, 90062, 10133,   100,   102,
            0,     0,     0,     0,     0])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0])


In [ ]:
class IMBDataset(Dataset):
  def __init__(self,Text,Label,tokenizer,max_len):
    self.Text = Text
    self.Label = Label
    self.tokenizer = tokenizer
    self.maxx_len = max_len

  def __len__(self):
    return len(self.Text)

  def __getitem__(self, item):
    text = str(self.Text[item])
    label = self.Label[item]
    encoding = tokenizer.encode_plus(
      simple_tex,
      max_length = 15,
      truncation = True,
      add_special_tokens = True,
      return_token_type_ids = False,
      padding = 'max_length',
      return_attention_mask = True,
      return_tensors = 'pt'
      )

    return {
          'Text': text,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'labels': torch.tensor(label, dtype=torch.long)
      }




In [ ]:
#DATA LOADER
def data_loader(df_destil,tokenizer,max_len,batch_size):
  dataset = IMBDataset(
      Text = df_destil.Text.to_numpy(),
      Label = df_destil.Label.to_numpy(),
      tokenizer = tokenizer,
      max_len = MAX_LEN
  )
  return DataLoader(dataset, batch_size = BATCH_SIZE, num_workers = 4)

In [ ]:
df_train, df_test = train_test_split(df_destil, test_size = 0.2, random_state=RANDOM_SEED)

train_data_loader = data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
class BERTModerador(nn.Module):

  def __init__(self,n_classes):
    super(BERTModerador,self).__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.dropout = nn.Dropout(p=0.3)
    self.linear = nn.Linear(
        self.bert.config.hidden_size,
        n_classes
    )

  def forward(self, input_ids, attention_mask):
    outputs = self.bert(
        input_ids = input_ids,
        attention_mask = attention_mask
    )
    pooled_output = outputs.pooler_output
    x = self.dropout(pooled_output)
    logits = self.linear(x)
    return logits


In [ ]:
model = BERTModerador(NCLASSES)
model = model.to(device)

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [ ]:
print(model)

BERTModerador(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
#ENTRENAMIENTO
EPOCHS = 7

from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
import torch.nn as nn

# Optimizador
optimizer = AdamW(
    model.parameters(),
    lr=2e-5
)


total_steps = len(train_data_loader) * EPOCHS

# Scheduler (control del learning rate)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

# Función de pérdida
loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:
#ENTRENAMIENTO BASE
def train_model(model,data_loader,loss_fn,optimizer,device,scheduler,n_examples):
  model = model.train()
  losses = []
  correct_predictions = 0
  for batch in data_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    outputs = model(input_ids, attention_mask)
    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, labels)
    correct_predictions += torch.sum(preds == labels)
    losses.append(loss.item())
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
  return correct_predictions.double()/n_examples, np.mean(losses)

#EVALUAR MODELO
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()
  losses = []
  correct_predictions = 0
  with torch.no_grad():
    for batch in data_loader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)
      outputs = model(input_ids, attention_mask)
      _, preds = torch.max(outputs, dim=1)
      loss = loss_fn(outputs, labels)
      correct_predictions += torch.sum(preds == labels)
      losses.append(loss.item())
  return correct_predictions.double()/n_examples, np.mean(losses)

In [ ]:
#ENTRENAMIENTO!!
for epoch in range(EPOCHS):
    print('Epoch {} de {}'.format(epoch+1, EPOCHS))
    print('------------------')
    train_acc, train_loss = train_model(
        model, train_data_loader, loss_fn, optimizer, device, scheduler, len(df_train)
    )
    test_acc, test_loss = eval_model(
        model, test_data_loader, loss_fn, device, len(df_test)
    )
    print('Entrenamiento: Loss: {}, accuracy: {}'.format(train_loss, train_acc))
    print('Validación: Loss: {}, accuracy: {}'.format(test_loss, test_acc))
    print('')

Epoch 1 de 7
------------------
Entrenamiento: Loss: 1.0512250691652298, accuracy: 0.4475
Validación: Loss: 1.0459495067596436, accuracy: 0.4466666666666667

Epoch 2 de 7
------------------
Entrenamiento: Loss: 1.0544589777787527, accuracy: 0.4391666666666667
Validación: Loss: 1.0459495067596436, accuracy: 0.4466666666666667

Epoch 3 de 7
------------------
Entrenamiento: Loss: 1.0556908090909323, accuracy: 0.44583333333333336
Validación: Loss: 1.0459495067596436, accuracy: 0.4466666666666667

Epoch 4 de 7
------------------
Entrenamiento: Loss: 1.0519769708315532, accuracy: 0.44583333333333336
Validación: Loss: 1.0459495067596436, accuracy: 0.4466666666666667

Epoch 5 de 7
------------------
Entrenamiento: Loss: 1.0531841193636258, accuracy: 0.45083333333333336
Validación: Loss: 1.0459495067596436, accuracy: 0.4466666666666667

Epoch 6 de 7
------------------
Entrenamiento: Loss: 1.0570638308922449, accuracy: 0.45083333333333336
Validación: Loss: 1.0459495067596436, accuracy: 0.446666

In [ ]:
id2label = {
    0: "clean",
    1: "toxic",
    2: "hate"
}

def classifyhate(review_text):
    model.eval()

    with torch.no_grad(): #manda a pytorch solo predizca
        encoding = tokenizer.encode_plus(
            review_text,
            max_length=MAX_LEN,
            truncation=True,
            add_special_tokens=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].to(device)
        attention_mask = encoding['attention_mask'].to(device)

        outputs = model(input_ids, attention_mask)

        #calculo las probabilidades
        probs = torch.softmax(outputs, dim=1).squeeze()

        preds = torch.argmax(outputs, dim=1) #busca el índice del valor más grande
        prediction = preds.item()

    print("\n".join(wrap(review_text)))
    print("\nProbabilidades:")
    for i, p in enumerate(probs):
        print(f"  {id2label[i]}: {p:.3f}")

    print(f"\nSentencia de moderador: {id2label[prediction]}")
    return prediction


In [ ]:
classifyhate("You are stupid and useless")
print("-" * 40)
classifyhate("I will kill you and your family")

You are stupid and useless

Probabilidades:
  clean: 0.454
  toxic: 0.347
  hate: 0.199

Sentencia de moderador: clean
----------------------------------------
I will kill you and your family

Probabilidades:
  clean: 0.456
  toxic: 0.344
  hate: 0.200

Sentencia de moderador: clean


0